In [5]:
import io
import requests
import supervision as sv
from PIL import Image
from rfdetr import RFDETRBase
from rfdetr.util.coco_classes import COCO_CLASSES

from rfdetr import RFDETRBase

# Cargar el modelo con pesos preentrenados
try:
    model = RFDETRBase(pretrained=True)  # Esto carga automáticamente los pesos
    print("✅ Modelo cargado correctamente")
except Exception as e:
    print(f"❌ Error al cargar el modelo: {e}")



Loading pretrain weights
✅ Modelo cargado correctamente


In [6]:
import cv2

cap = cv2.VideoCapture(0)
while True:
    success, frame = cap.read()
    if not success:
        break

    detections = model.predict(frame, threshold=0.5)

    labels = [
        f"{COCO_CLASSES[class_id]} {confidence:.2f}"
        for class_id, confidence
        in zip(detections.class_id, detections.confidence)
    ]

    annotated_frame = frame.copy()
    annotated_frame = sv.BoxAnnotator().annotate(annotated_frame, detections)
    annotated_frame = sv.LabelAnnotator().annotate(annotated_frame, detections, labels)

    cv2.imshow("Webcam", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Track every detection

In [8]:
import cv2
import time
import numpy as np
from datetime import datetime
from collections import defaultdict, deque

# Inicializar el modelo con resolución moderada para equilibrar velocidad y precisión
model = RFDETRBase(resolution=448)

# Inicializar webcam
cap = cv2.VideoCapture(0)
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Estructuras de datos para almacenar detecciones
detection_history = {
    'objects': defaultdict(int),           # Contador total por clase
    'timestamps': [],                      # Timestamps de cada frame
    'detections_per_frame': [],            # Número de detecciones por frame
    'confidence_history': defaultdict(list) # Historial de confianza por clase
}

# Cola para mantener las últimas 100 detecciones (formato de registro)
recent_detections = deque(maxlen=100)

# Para calcular FPS
prev_time = time.time()
fps_history = deque(maxlen=30)  # Para calcular FPS promedio

# Umbral de confianza para detecciones
confidence_threshold = 0.4

try:
    while True:
        # Leer frame de la webcam
        ret, frame = cap.read()
        if not ret:
            break

        # Registrar timestamp
        current_time = datetime.now()
        timestamp = current_time.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        detection_history['timestamps'].append(timestamp)

        # Convertir frame de BGR a RGB para el modelo
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Ejecutar inferencia
        detections = model.predict(rgb_frame, threshold=confidence_threshold)

        # Registrar número de detecciones en este frame
        num_detections = len(detections.confidence)
        detection_history['detections_per_frame'].append(num_detections)

        # Procesar y almacenar detecciones
        frame_detections = []
        for i, (class_id, confidence, box) in enumerate(zip(
                detections.class_id,
                detections.confidence,
                detections.xyxy)):

            class_name = COCO_CLASSES[class_id]

            # Actualizar contador de objetos
            detection_history['objects'][class_name] += 1

            # Guardar historial de confianza
            detection_history['confidence_history'][class_name].append(float(confidence))

            # Crear registro detallado de esta detección
            detection_record = {
                'timestamp': timestamp,
                'class_id': int(class_id),
                'class_name': class_name,
                'confidence': float(confidence),
                'box': box.tolist(),
                'frame_position': i
            }

            # Añadir a la cola de detecciones recientes
            recent_detections.append(detection_record)
            frame_detections.append(detection_record)

        # Preparar etiquetas para visualización
        labels = [
            f"{COCO_CLASSES[class_id]} {confidence:.2f}"
            for class_id, confidence
            in zip(detections.class_id, detections.confidence)
        ]

        # Anotar frame
        annotated_frame = box_annotator.annotate(frame, detections)
        annotated_frame = label_annotator.annotate(annotated_frame, detections, labels)

        # Calcular y mostrar FPS
        current_time = time.time()
        fps = 1 / (current_time - prev_time)
        fps_history.append(fps)
        avg_fps = sum(fps_history) / len(fps_history)
        prev_time = current_time

        # Mostrar estadísticas en el frame
        cv2.putText(annotated_frame, f"FPS: {avg_fps:.1f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Mostrar top 3 objetos detectados
        top_objects = sorted(detection_history['objects'].items(),
                             key=lambda x: x[1], reverse=True)[:3]
        y_pos = 70
        for obj_name, count in top_objects:
            cv2.putText(annotated_frame, f"{obj_name}: {count}", (10, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            y_pos += 30

        # Mostrar número de objetos en la escena actual
        current_objects = {}
        for det in frame_detections:
            current_objects[det['class_name']] = current_objects.get(det['class_name'], 0) + 1

        cv2.putText(annotated_frame, f"Objetos actuales: {len(current_objects)}",
                    (10, y_pos + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Mostrar el resultado
        cv2.imshow("RF-DETR Webcam Detection con Registro", annotated_frame)

        # Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Detenido por el usuario")
finally:
    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

    # Mostrar resumen de detecciones
    print("\n===== RESUMEN DE DETECCIONES =====")
    print(f"Total de frames procesados: {len(detection_history['timestamps'])}")
    print(f"Promedio de detecciones por frame: {np.mean(detection_history['detections_per_frame']):.2f}")
    print("\nTop 5 objetos detectados:")

    for class_name, count in sorted(detection_history['objects'].items(), key=lambda x: x[1], reverse=True)[:5]:
        avg_confidence = np.mean(detection_history['confidence_history'][class_name])
        print(f"  - {class_name}: {count} detecciones (confianza promedio: {avg_confidence:.2f})")

    print(f"\nÚltimas 5 detecciones:")
    for det in list(recent_detections)[-5:]:
        print(f"  - {det['timestamp']}: {det['class_name']} ({det['confidence']:.2f})")

Loading pretrain weights

===== RESUMEN DE DETECCIONES =====
Total de frames procesados: 97
Promedio de detecciones por frame: 1.85

Top 5 objetos detectados:
  - person: 97 detecciones (confianza promedio: 0.97)
  - cell phone: 33 detecciones (confianza promedio: 0.63)
  - remote: 13 detecciones (confianza promedio: 0.49)
  - cup: 11 detecciones (confianza promedio: 0.91)
  - book: 8 detecciones (confianza promedio: 0.44)

Últimas 5 detecciones:
  - 2025-04-02 18:07:32.417: person (0.95)
  - 2025-04-02 18:07:32.417: book (0.45)
  - 2025-04-02 18:07:33.484: person (0.95)
  - 2025-04-02 18:07:34.553: person (0.95)
  - 2025-04-02 18:07:34.553: book (0.45)


Detect unique objects

In [9]:
import os
import json

# Inicializar el modelo
model = RFDETRBase()

# Inicializar la captura de video
cap = cv2.VideoCapture(0)

# Directorio para guardar los datos
output_dir = "objetos_detectados"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Archivo para el registro JSON
json_file = os.path.join(output_dir, "registro.json")

# Cargar registro existente si existe
objects_registry = {}
if os.path.exists(json_file):
    with open(json_file, 'r') as f:
        objects_registry = json.load(f)

while True:
    success, frame = cap.read()
    if not success:
        break
    
    # Realizar detección de objetos
    detections = model.predict(frame, threshold=0.5)
    
    # Procesar cada detección
    for i, (class_id, confidence, bbox) in enumerate(zip(detections.class_id, detections.confidence, detections.xyxy)):
        # Obtener el nombre de la clase
        class_name = COCO_CLASSES[class_id]
        
        # Verificar si es un objeto único (no detectado anteriormente)
        # o si la detección actual tiene mayor confianza
        if class_name not in objects_registry or confidence > objects_registry[class_name]["confidence"]:
            # Extraer la región de interés (ROI)
            x1, y1, x2, y2 = map(int, bbox)
            roi = frame[y1:y2, x1:x2]
            
            # Guardar la imagen recortada
            img_path = os.path.join(output_dir, f"{class_name}.jpg")
            cv2.imwrite(img_path, roi)
            
            # Actualizar o crear registro
            objects_registry[class_name] = {
                "confidence": float(confidence),
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "image_path": img_path
            }
            
            # Guardar registro actualizado
            with open(json_file, 'w') as f:
                json.dump(objects_registry, f, indent=4)
            
            print(f"Objeto actualizado/guardado: {class_name} (Confianza: {confidence:.2f})")
    
    # Crear etiquetas para visualización
    labels = [
        f"{COCO_CLASSES[class_id]} {confidence:.2f}"
        for class_id, confidence
        in zip(detections.class_id, detections.confidence)
    ]
    
    # Anotar el frame con detecciones
    annotated_frame = frame.copy()
    annotated_frame = sv.BoxAnnotator().annotate(annotated_frame, detections)
    annotated_frame = sv.LabelAnnotator().annotate(annotated_frame, detections, labels)
    
    # Mostrar el número de objetos únicos detectados
    cv2.putText(
        annotated_frame, 
        f"Objetos únicos: {len(objects_registry)}", 
        (10, 30), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1, 
        (0, 255, 0), 
        2
    )
    
    # Mostrar el frame anotado
    cv2.imshow("Webcam", annotated_frame)
    
    # Salir si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()

Loading pretrain weights
Objeto actualizado/guardado: person (Confianza: 0.96)
Objeto actualizado/guardado: person (Confianza: 0.96)
Objeto actualizado/guardado: person (Confianza: 0.96)
Objeto actualizado/guardado: person (Confianza: 0.96)
Objeto actualizado/guardado: person (Confianza: 0.97)
Objeto actualizado/guardado: apple (Confianza: 0.89)
Objeto actualizado/guardado: apple (Confianza: 0.91)
Objeto actualizado/guardado: apple (Confianza: 0.92)
Objeto actualizado/guardado: cup (Confianza: 0.57)
Objeto actualizado/guardado: cup (Confianza: 0.94)
Objeto actualizado/guardado: cup (Confianza: 0.96)
Objeto actualizado/guardado: remote (Confianza: 0.89)
Objeto actualizado/guardado: cell phone (Confianza: 0.69)
Objeto actualizado/guardado: cell phone (Confianza: 0.71)


Specific detections

In [10]:
import cv2
import time
import supervision as sv
from datetime import datetime
import os
from collections import defaultdict
from rfdetr import RFDETRBase
from rfdetr.util.coco_classes import COCO_CLASSES
import matplotlib
# Configurar matplotlib para no usar GUI
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Configuración de directorios
os.makedirs("detecciones", exist_ok=True)
os.makedirs("estadisticas", exist_ok=True)

# Definir objetos de interés con configuraciones personalizadas
OBJETOS_DE_INTERES = {
    "cup": {"prioridad": "media", "color": (0, 0, 255)},
    "knife": {"prioridad": "alta", "color": (255, 165, 0)},
    "book": {"prioridad": "baja", "color": (255, 0, 255)},
}

# Inicializar modelo
model = RFDETRBase(resolution=448)

# Inicializar webcam
cap = cv2.VideoCapture(0)
box_annotator = sv.BoxAnnotator()

# Variables para estadísticas y seguimiento (solo para objetos de interés)
estadisticas = {
    "total_detecciones": defaultdict(int),
    "registros_generados": defaultdict(int),
    "confianza_promedio": defaultdict(list),
    "historial_tiempo": [],
    "historial_detecciones": []
}

# Tiempo mínimo entre registros del mismo tipo (en segundos)
tiempo_entre_registros = 5
ultimo_registro = defaultdict(float)

# Funciones de utilidad
def registrar_deteccion(frame, objeto, confianza, bbox):
    """Guarda una captura del frame con la detección destacada"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"detecciones/{objeto}_{timestamp}.jpg"

    # Destacar el objeto detectado
    highlighted = frame.copy()
    x1, y1, x2, y2 = [int(c) for c in bbox]
    color = OBJETOS_DE_INTERES[objeto]["color"]
    cv2.rectangle(highlighted, (x1, y1), (x2, y2), color, 4)
    cv2.putText(highlighted, f"{objeto} ({confianza:.2f})", (x1, y1-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Añadir información de la detección
    info_text = f"DETECCIÓN: {objeto} | Prioridad: {OBJETOS_DE_INTERES[objeto]['prioridad'].upper()} | Confianza: {confianza:.2f} | {timestamp}"
    cv2.putText(highlighted, info_text, (10, frame.shape[0]-20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imwrite(filename, highlighted)
    return filename

# Bucle principal de captura y procesamiento
try:
    prev_time = time.time()
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()
        fps = 1 / (current_time - prev_time)
        prev_time = current_time

        # Timestamp para este frame
        timestamp = datetime.now()

        # Ejecutar la detección
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detections = model.predict(rgb_frame, threshold=0.45)

        # Filtrar solo los objetos de interés
        filtered_indexes = []
        filtered_boxes = []
        filtered_class_ids = []
        filtered_confidences = []
        filtered_labels = []
        filtered_colors = []

        objetos_en_escena = defaultdict(int)
        total_objetos_interes = 0

        # Filtrar detecciones para incluir solo objetos de interés
        for i, (class_id, confidence, box) in enumerate(zip(
                detections.class_id,
                detections.confidence,
                detections.xyxy)):

            class_name = COCO_CLASSES[class_id]

            # Solo procesar si es un objeto de interés
            if class_name in OBJETOS_DE_INTERES:
                filtered_indexes.append(i)
                filtered_boxes.append(box)
                filtered_class_ids.append(class_id)
                filtered_confidences.append(confidence)

                # Incrementar contador para este objeto
                objetos_en_escena[class_name] += 1
                total_objetos_interes += 1

                # Añadir a estadísticas
                estadisticas["total_detecciones"][class_name] += 1
                estadisticas["confianza_promedio"][class_name].append(float(confidence))

                # Preparar etiqueta y color para visualización
                color = OBJETOS_DE_INTERES[class_name]["color"]
                prioridad = OBJETOS_DE_INTERES[class_name]["prioridad"]
                label = f"{class_name} [{prioridad.upper()}]"
                filtered_labels.append(label)
                filtered_colors.append(color)

                # Verificar si debemos registrar esta detección
                tiempo_actual = time.time()
                if tiempo_actual - ultimo_registro[class_name] > tiempo_entre_registros:
                    # Registrar detección con captura de pantalla
                    imagen_deteccion = registrar_deteccion(frame, class_name, confidence, box)

                    # Actualizar tiempo del último registro
                    ultimo_registro[class_name] = tiempo_actual
                    estadisticas["registros_generados"][class_name] += 1

                    print(f"Objeto de interés detectado: {class_name} - Imagen guardada: {imagen_deteccion}")

        # Actualizar historial solo si hay objetos de interés
        if len(filtered_indexes) > 0:
            estadisticas["historial_tiempo"].append(timestamp)
            estadisticas["historial_detecciones"].append(total_objetos_interes)

        # Crear un frame limpio para visualización (sin anotaciones)
        clean_frame = frame.copy()

        # Dibujar solo los objetos de interés filtrados manualmente
        for i, (box, label, color) in enumerate(zip(filtered_boxes, filtered_labels, filtered_colors)):
            x1, y1, x2, y2 = [int(c) for c in box]
            # Dibujar rectángulo
            cv2.rectangle(clean_frame, (x1, y1), (x2, y2), color, 2)
            # Añadir etiqueta
            cv2.putText(clean_frame, label, (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Mostrar FPS
        cv2.putText(clean_frame, f"FPS: {fps:.1f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Mostrar solo el frame con las detecciones
        cv2.imshow("Monitoreo de Objetos", clean_frame)

        # Salir con 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Detenido por el usuario")
finally:
    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

    # Si no hay detecciones, mostrar mensaje y salir
    if not estadisticas["total_detecciones"]:
        print("No se detectaron objetos de interés durante la sesión.")
        exit()

    # Guardar estadísticas finales
    timestamp_final = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Generar gráfico de resumen (solo objetos de interés)
    plt.figure(figsize=(15, 10))

    # Gráfico 1: Total de detecciones
    plt.subplot(2, 2, 1)
    objetos = list(estadisticas["total_detecciones"].keys())
    valores = list(estadisticas["total_detecciones"].values())
    plt.bar(objetos, valores)
    plt.title('Total de Detecciones de Objetos de Interés')
    plt.xticks(rotation=45)

    # Gráfico 2: Registros generados
    plt.subplot(2, 2, 2)
    objetos = list(estadisticas["registros_generados"].keys())
    valores = list(estadisticas["registros_generados"].values())
    plt.bar(objetos, valores, color='orange')
    plt.title('Imágenes Registradas por Objeto')
    plt.xticks(rotation=45)

    # Gráfico 3: Confianza promedio
    plt.subplot(2, 2, 3)
    objetos = []
    valores = []
    for obj, confianzas in estadisticas["confianza_promedio"].items():
        if confianzas:
            objetos.append(obj)
            valores.append(sum(confianzas) / len(confianzas))
    plt.bar(objetos, valores, color='green')
    plt.title('Confianza Promedio por Objeto')
    plt.xticks(rotation=45)

    # Gráfico 4: Timeline de detecciones
    plt.subplot(2, 2, 4)
    plt.plot(estadisticas["historial_tiempo"], estadisticas["historial_detecciones"])
    plt.title('Objetos de Interés Detectados en el Tiempo')
    plt.xticks(rotation=45)

    plt.tight_layout()
    # Guardar el gráfico sin mostrarlo
    plt.savefig(f"estadisticas/resumen_{timestamp_final}.png")
    plt.close()  # Cerrar la figura después de guardarla

    print("\n===== RESUMEN DE LA SESIÓN =====")
    total_tiempo = 0
    if estadisticas["historial_tiempo"]:
        total_tiempo = (datetime.now() - estadisticas["historial_tiempo"][0]).total_seconds()

    print(f"Duración total: {total_tiempo:.2f} segundos")
    print(f"Total de objetos de interés detectados: {sum(estadisticas['total_detecciones'].values())}")
    print(f"Total de imágenes registradas: {sum(estadisticas['registros_generados'].values())}")
    print(f"Objetos de interés detectados:")
    for obj, count in sorted(estadisticas["total_detecciones"].items(), key=lambda x: x[1], reverse=True):
        print(f"  - {obj}: {count} veces")

    print(f"\nInforme guardado en: estadisticas/resumen_{timestamp_final}.png")
    print(f"Imágenes guardadas en: detecciones/")

Loading pretrain weights
Objeto de interés detectado: book - Imagen guardada: detecciones/book_20250402_181903.jpg
Objeto de interés detectado: cup - Imagen guardada: detecciones/cup_20250402_181936.jpg
Objeto de interés detectado: cup - Imagen guardada: detecciones/cup_20250402_181942.jpg
Objeto de interés detectado: cup - Imagen guardada: detecciones/cup_20250402_182048.jpg

===== RESUMEN DE LA SESIÓN =====
Duración total: 181.46 segundos
Total de objetos de interés detectados: 11
Total de imágenes registradas: 4
Objetos de interés detectados:
  - cup: 10 veces
  - book: 1 veces

Informe guardado en: estadisticas/resumen_20250402_182201.png
Imágenes guardadas en: detecciones/
